In [1]:
import scanpy as sc 

In [2]:
cd /nfs/team298/sm54/BoneAtlasProject/src/processing

/nfs/team298/sm54/BoneAtlasProject/src/processing


In [3]:
#!/usr/bin/env python
# coding: utf-8

"""
Stage 1: Read metadata, select samples, generate per-sample .h5ad and merged AnnData.
Logs to <output_dir>/logs/generate_h5ad.log.
"""

import argparse
import logging
import os
import pandas as pd
import scanpy as sc
from helper_functions import generate_h5ads
import os
import scanpy as sc
import numpy as np
from scipy.stats import median_abs_deviation
import pandas as pd
import celltypist
import scrublet as scr
import scipy.stats
import statsmodels.stats.multitest as smm
import matplotlib.pyplot as plt
from utils import ensure_dir
import matplotlib.pyplot as plt
import statsmodels.stats.multitest as smm
# Utility Functions

In [4]:
def is_outlier(adata, metric: str, nmads: int):
    """
    Identify outliers in an AnnData.obs metric using Median Absolute Deviation (MAD).
    """
    M = adata.obs[metric]
    mad = median_abs_deviation(M)
    return (M < np.median(M) - nmads * mad) | (M > np.median(M) + nmads * mad)
    
def is_upper_outlier(adata, metric: str, nmads: int):
    """
    Flag cells whose obs[metric] is more than nmads*MAD above the median.
    """
    M   = adata.obs[metric]
    mad = median_abs_deviation(M)
    thresh = np.median(M) + nmads * mad
    return M > thresh



In [5]:
def generate_h5ads_from_mtx(
    sample_list,
    raw_dir,
    out_dir,
    compression='gzip',
    merged_fname='merged.h5ad',
):
    ensure_dir(out_dir)

    import pandas as pd
    from scipy.io import mmread

    adatas = []
    processed = []

    for sanger in sample_list:

        mat  = os.path.join(raw_dir, f"{sanger}_matrix.mtx.gz")
        feat = os.path.join(raw_dir, f"{sanger}_features.tsv.gz")
        bar  = os.path.join(raw_dir, f"{sanger}_barcodes.tsv.gz")

        if not all(os.path.exists(x) for x in [mat,feat,bar]):
            print("Skipping",sanger)
            continue

        print("→ Loading",sanger)

        X = mmread(mat).tocsr().T
        genes = pd.read_csv(feat,sep="\t",header=None)[1].astype(str).values
        bars  = pd.read_csv(bar,sep="\t",header=None)[0].astype(str).values

        ad = sc.AnnData(X)
        ad.var_names = genes
        ad.obs_names = bars

        ad.obs["barcode_raw"]=ad.obs_names
        ad.obs["Sanger_ID"]=sanger
        ad.obs["Cell_ID"]=ad.obs["barcode_raw"]+"-"+sanger
        ad.obs_names=ad.obs["Cell_ID"]

        ad.var["SYMBOL"]=ad.var_names
        ad.var_names_make_unique()
        ad.var["mt"]=ad.var_names.str.startswith("MT-")
        ad.var["rb"]=ad.var_names.str.startswith(("RPS","RPL"))

        sc.pp.calculate_qc_metrics(ad,qc_vars=["mt","rb"],inplace=True,log1p=True)
        ad.obs["total_counts_outlier"]=is_outlier(ad,"log1p_total_counts",5)
        ad.obs["gene_counts_outlier"]=is_outlier(ad,"log1p_n_genes_by_counts",5)
        ad.obs["mt_outlier"]=is_upper_outlier(ad,"log1p_total_counts_mt",3)

        # -------- THE ONLY FIX REQUIRED --------
        del ad.obs["Cell_ID"]            # avoid duplicate index name conflict
        ad.obs_names.name="Cell_ID"      # keep index with same meaning
        # ---------------------------------------

        adatas.append(ad)
        processed.append(sanger)

    print("\n→ merging…")
    merged = sc.concat(adatas,join="outer",label="Sanger_ID",keys=processed,index_unique="-")

    out=os.path.join(out_dir,merged_fname)
    merged.write_h5ad(out,compression=compression)

    print("✓ Saved:",out)
    return merged


In [6]:
cd /lustre/scratch127/cellgen/cellgeni/tickets/tic-2943/data

/lustre/scratch127/cellgen/cellgeni/tickets/tic-2943/data


In [7]:
raw_dir="/lustre/scratch127/cellgen/cellgeni/tickets/tic-2943/data"
sample_list = sorted({f.replace("_matrix.mtx.gz","") for f in os.listdir(raw_dir) if f.endswith("_matrix.mtx.gz")})


In [22]:
merge = generate_h5ads_from_mtx(
    sample_list,
    raw_dir=raw_dir,
    out_dir="/lustre/scratch124/cellgen/haniffa/users/sm54/data/Bone_Atlas/"
)

→ Loading GSM4080376_GW10C_01NC
→ Loading GSM4080377_GW10C_02


KeyboardInterrupt: 

In [10]:
merge

AnnData object with n_obs × n_vars = 1112228 × 45068
    obs: 'barcode_raw', 'Sanger_ID', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'log1p_total_counts_rb', 'pct_counts_rb', 'total_counts_outlier', 'gene_counts_outlier', 'mt_outlier'

In [13]:
conditions = [
    (merge.obs['total_counts_outlier']== True),
    (merge.obs['gene_counts_outlier'] == True),
     (merge.obs['mt_outlier'] == True),
    (
    (merge.obs['gene_counts_outlier']== False) & (merge.obs['gene_counts_outlier'] == False) & (merge.obs['mt_outlier']==False))
   
]
values = ['Low_total_count', 'Low_nFeature', "High_Mito",'Pass']

In [14]:
merge.obs['QC'] = np.select(conditions, values, default="Pass")

In [34]:
merge.obs['Sanger_ID'].value_counts()

Sanger_ID
GSM4080451_GW8S_02NC     25282
GSM4080422_GW19C         25173
GSM4080450_GW8S_02       24874
GSM4080413_GW17C_01      24683
GSM4080423_GW19L         24375
                         ...  
GSM4080403_GW14T_NC       7045
GSM4080384_GW10T_02NC     7026
GSM4080382_GW10T_01NC     4845
GSM4080376_GW10C_01NC     4724
GSM4080379_GW10L_01NC     4615
Name: count, Length: 78, dtype: int64

In [17]:
adata=merge[merge.obs['QC']=="Pass"].copy()

In [20]:
adata=adata[adata.obs['total_counts']>500]

In [21]:
adata

View of AnnData object with n_obs × n_vars = 1037632 × 45068
    obs: 'barcode_raw', 'Sanger_ID', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'log1p_total_counts_rb', 'pct_counts_rb', 'total_counts_outlier', 'gene_counts_outlier', 'mt_outlier', 'QC'

In [23]:
adata.write_h5ad('/lustre/scratch124/cellgen/haniffa/users/sm54/data/Bone_Atlas/Zhang_spinal_cord_filtered.h5ad')

... storing 'QC' as categorical


In [36]:
  subset = adata[adata.obs["Sanger_ID"] == 'GSM4080451_GW8S_02NC'].copy()

In [38]:
sc.pp.normalize_total(subset, target_sum=10000)
sc.pp.log1p(subset)

In [40]:
result = celltypist.annotate(
            subset,
            model='/nfs/team298/sm54/BoneAtlasProject/metadata/celltypist_models/Human_WholeEmbryo_Public.pkl',
            majority_voting=True
        )


🔬 Input data has 18545 cells and 45068 genes
🔗 Matching reference genes in the model
🧬 7296 features used for prediction
⚖️ Scaling input data
🖋️ Predicting labels
✅ Prediction done!
👀 Can not detect a neighborhood graph, will construct one before the over-clustering
⛓️ Over-clustering input data with resolution set to 10
🗳️ Majority voting the predictions
✅ Majority voting done!


In [48]:
def annotate_celltypist_merged(
    adata,
    model_path: str,
    out_dir: str = None,
    target_sum: float = 1e4,
):

    model = celltypist.models.Model.load(model=model_path)
    model_name = os.path.basename(model_path).replace(".pkl","")

    pred_list = []

    # --- ensure destination columns exist & are non-categorical ---
    for col in [f"celltypist_{model_name}_pred",
                f"celltypist_{model_name}_majority"]:
        if col not in adata.obs:
            adata.obs[col] = None
        adata.obs[col] = adata.obs[col].astype("object")  # prevents category errors

    for sample in adata.obs["Sanger_ID"].unique():
        
        subset = adata[adata.obs["Sanger_ID"] == sample].copy()

        sc.pp.normalize_total(subset, target_sum=target_sum)
        sc.pp.log1p(subset)

        result = celltypist.annotate(
            subset,
            model=model,
            majority_voting=True
        )

        labels = result.predicted_labels.reset_index()
        labels["Sanger_ID"] = sample
        pred_list.append(labels)

        # ---- attach to main AnnData ----
        adata.obs.loc[subset.obs_names,
            f"celltypist_{model_name}_pred"
        ] = labels["predicted_labels"].values

        adata.obs.loc[subset.obs_names,
            f"celltypist_{model_name}_majority"
        ] = labels["majority_voting"].values

    merged_pred = pd.concat(pred_list, axis=0)

    if out_dir is not None:
        merged_pred.to_csv(f"{out_dir}/Celltypist_predictions.csv", index=False)

    print("✓ Completed — returning annotated AnnData + merged predictions")
    return adata, merged_pred


In [49]:
adata_ct, prob, pred = annotate_celltypist_merged(
  adata,
    model_path="/nfs/team298/sm54/BoneAtlasProject/metadata/celltypist_models/Human_WholeEmbryo_Public.pkl",
    out_dir="/lustre/scratch124/cellgen/haniffa/users/sm54/data/Bone_Atlas/")

🔬 Input data has 3616 cells and 45068 genes
🔗 Matching reference genes in the model
🧬 7296 features used for prediction
⚖️ Scaling input data
🖋️ Predicting labels
✅ Prediction done!
👀 Can not detect a neighborhood graph, will construct one before the over-clustering
⛓️ Over-clustering input data with resolution set to 5
🗳️ Majority voting the predictions
✅ Majority voting done!
🔬 Input data has 13645 cells and 45068 genes
🔗 Matching reference genes in the model
🧬 7296 features used for prediction
⚖️ Scaling input data
🖋️ Predicting labels
✅ Prediction done!
👀 Can not detect a neighborhood graph, will construct one before the over-clustering
⛓️ Over-clustering input data with resolution set to 10
🗳️ Majority voting the predictions
✅ Majority voting done!
🔬 Input data has 6303 cells and 45068 genes
🔗 Matching reference genes in the model
🧬 7296 features used for prediction
⚖️ Scaling input data
🖋️ Predicting labels
✅ Prediction done!
👀 Can not detect a neighborhood graph, will construct 

✓ Completed — returning annotated AnnData + merged predictions


ValueError: not enough values to unpack (expected 3, got 2)

In [56]:
adata.obs['celltypist_Human_WholeEmbryo_Public_majority'].value_counts()[120:190]

celltypist_Human_WholeEmbryo_Public_majority
Coelomic epithelial cells                    31
AGM endothelial cells                        31
CD5- mature B                                30
Definitive erythroblast                      29
Brain erythrocytes                           28
Myocardial fibroblast                        27
Cycling DC                                   27
Cerebellum neuronal IPC                      24
Striatum glioblast                           24
Forebrain radial glia                        24
MFAP5+ fibroblast                            23
CD8AA                                        22
Chondroprogenitors                           22
HIC1+ mesenchymal cells                      21
B1 cells                                     21
Early supporting-like PAX8+ cells            21
Cortex neuron                                19
Early retinal glia                           16
Smooth muscle progenitors                    15
Hypothalamus radial glia                   

In [57]:
adata.write_h5ad('/lustre/scratch124/cellgen/haniffa/users/sm54/data/Bone_Atlas/Zhang_spinal_cord_filtered_normalised_with_anno.h5ad')

... storing 'celltypist_Human_WholeEmbryo_Public_pred' as categorical
... storing 'celltypist_Human_WholeEmbryo_Public_majority' as categorical
